In [1]:
import os,json,io
print ('Getting base directory... ', end='')
base = os.getcwd()
print ('Done\nSetting the base at', base)

Getting base directory... Done
Setting the base at /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis


In [2]:
from bs4 import BeautifulSoup as bsp
print ('Changing to the commentary db folder', end='')
os.chdir(os.path.join(base,'commentaries'))
print (os.getcwd(),'... Done')

print('Collecting all matches in current folder...', end='')
matches = [f for f in os.scandir(os.getcwd()) if f.is_dir() and f.name.isdigit()];
print(' Done')

Changing to the commentary db folder/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries ... Done


In [ ]:
#test
matches

In [3]:
#putting frequently used text in variable, helpful in IDE's with auto completion
t_commentary = 'commentary'
t_items = 'items'
t_shorttext = 'shortText'
t_postText = 'postText'
t_text = 'text'
t_preText = 'preText'

#functions I'll need
#-----------------------------------------get_match_details-----------------------------------------------------
def get_match_details(match_id):
    import requests
    r = requests.get(f'http://www.espncricinfo.com/series/engine/match/{match_id}.json')
    return r.json()
#---------------------------------------------------------------------------------------------------------------
def get_required_innings_value(country):
    pass
    
def get_vader_sentence_score(sentence_string):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

    scores = SentimentIntensityAnalyzer()

    return scores.polarity_scores(sentence_string)

#----------------------------------------------------------------------------------------------------------------------------
def get_all_text_fromjsonfile(data):
    alltext = ''
    bslash = '\\'
    for item in data[t_commentary][t_items]:
        alltext += item[t_preText] + item[t_postText]
    return bsp(alltext, 'lxml').text
#--------------------------------------------------------------------------------------------------------------------
def get_player_score_fromjson(data):
    import nltk, io
    from nltk.tokenize import sent_tokenize, word_tokenize
    from nltk.corpus import stopwords

    def without_stopwords(word_list):
        stopset = set(stopwords.words('english'))
        return [x for x in word_list if x not in stopset]

    players = dict()
#     personal_comm = dict() # run this externally and comment this in the final run
    with io.open(os.path.join(base,'player_names.json'), 'r') as fp:
        players = json.load(fp)
#         print('\n', players,'\n')
        for sent in nltk.sent_tokenize(get_all_text_fromjsonfile(data)):
#             print (sent)
            for chunk in nltk.ne_chunk(nltk.pos_tag(without_stopwords(nltk.word_tokenize(sent)))):
                if hasattr(chunk, 'label') and (chunk.label() == 'PERSON'):
                    playerFromSent = ' '.join(c[0] for c in chunk)
#                     print(f'\nworking with {playerFromSent}\n')
                    for player in players.keys():
                        if playerFromSent.lower() in players[player]:
#                             print (playerFromSent,'found\n')
                            personal_comm.setdefault(player.lower(),[]).append([get_vader_sentence_score(sent)['compound'],sent])
    return personal_comm

# <span style="color:red;">Let's Begin!</span>

In [4]:
personal_comm = dict()
for match in matches:
    os.chdir(match.path);
    #match_details = get_match_details(match.name)
    print ("moving to "+match.path);
    innings = [f.path for f in os.scandir(match.path) if f.is_dir()]
    for single_innings in innings:
        os.chdir(single_innings);
        print ("moving to "+single_innings);
        for jsonfile in os.listdir():
            if 'comm_part' in jsonfile:
                print("opening " + jsonfile)
                with io.open(jsonfile, 'r') as file:
                    data = json.load(file)
                    get_player_score_fromjson(data)
                    
#                     print(metadata)
personal_comm

{'mashrafe mortaza': [[0.0,
   'Mashrafe Mortaza gives his team a pep talk before the huddle disperses.'],
  [0.0, '10.17am Toss - Bangladesh opt to field, says Mashrafe Mortaza.'],
  [-0.5106,
   'Bangladesh XI: 1 Tamim Iqbal, 2 Soumya Sarkar, 3 Mushfiqur Rahim (wk), 4 Mohammad Mithun, 5 Mahmudullah, 6, Mossaddek Hussain, 7  Sabbir Rahman, 8 Mohammed Saifuddin, 9 Mehidy Hasan Miraz, 10 Mashrafe Mortaza (capt), 11 Mustafizur Rahman\n\nWest Indies XI: 1 Sunil Ambris, 2 Shai Hope (wk), 3 Darren Bravo, 4 Roston Chase, 5 Jonathan Carter, 6 Jason Holder (capt), 7 Fabian Allen, 8 Ashley Nurse, 9 Kemar Roach, 10 Ramon Reiffer, 11 Shannon Gabriel\n\nNitol: "That makes it three consecutive ODI final for Bangladesh without Shakib due to injury"\n\nSherwin graham: "Goodmornng, I think windies should try something different, take Bangladesh out of their comfort zone, the tigers have beaten windies twice batting second.'],
  [0.7783,
   "Same field as Mashrafe, pretty much.A little bit of nibble of

In [8]:
match.path

'/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168509'

# Next Step
#### Create a function that takes in commetary json and spits out a dictionary that contains player names and commentary sentences about them
##### Better yet, make a function to evaluate a sentence first, then put a candidate sentence through it and get a result
### Okay, for simplicity let's work with <i style="color : red">postText</i>, <i style="color : red">preText</i> and give a scoring based on that
    - have to add the save_player_names to this module

In [ ]:
os.getcwd()

In [ ]:
match_details

In [23]:
with open(os.path.join(base,'result-json2.json'), 'w') as fp:
    json.dump(personal_comm,fp)

In [9]:
os.getcwd()

'/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168509/innings02'

In [17]:
result = dict()
# sumofpoints = 0
for key in personal_comm.keys():
    sumofpoints = 0
    for item in personal_comm[key]:
        sumofpoints += item[0]
#     print (key, sumofpoints, len(personal_comm[key]))
    result[key] = sumofpoints/len(personal_comm[key])

In [27]:
result

{'mashrafe mortaza': 0.22858319999999988,
 'shakib al hasan': 0.08590555555555557,
 'liton das': 0.10149651162790697,
 'soumya sarkar': 0.15051882352941176,
 'mushfiqur rahim': 0.09308783783783783,
 'mehidy hasan': 0.1963518518518519,
 'rubel hossain': 0.02547333333333333,
 'mahmudullah': 0.07007692307692306,
 'tamim iqbal': 0.11337163120567378,
 'imrul kayes': 0.13948260869565218,
 'mohammad mithun': 0.12819142857142862,
 'mustafizur rahman': 0.11148928571428572,
 'najmul hossain shanto': 0.02205,
 'anamul haque': -0.14991333333333337,
 'mohammad saifuddin': 0.15109333333333336,
 'nasir hossain': -0.016720000000000002,
 'taskin ahmed': 0.046271428571428574}

In [18]:
import operator
sorted_list = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [29]:
personal_comm.keys()

dict_keys(['mashrafe mortaza', 'shakib al hasan', 'liton das', 'soumya sarkar', 'mushfiqur rahim', 'mehidy hasan', 'rubel hossain', 'mahmudullah', 'tamim iqbal', 'imrul kayes', 'mohammad mithun', 'mustafizur rahman', 'najmul hossain shanto', 'anamul haque', 'mohammad saifuddin', 'nasir hossain', 'taskin ahmed'])

In [25]:
sorted_list

[('mashrafe mortaza', 0.22858319999999988),
 ('mehidy hasan', 0.1963518518518519),
 ('mohammad saifuddin', 0.15109333333333336),
 ('soumya sarkar', 0.15051882352941176),
 ('imrul kayes', 0.13948260869565218),
 ('mohammad mithun', 0.12819142857142862),
 ('tamim iqbal', 0.11337163120567378),
 ('mustafizur rahman', 0.11148928571428572),
 ('liton das', 0.10149651162790697),
 ('mushfiqur rahim', 0.09308783783783783),
 ('shakib al hasan', 0.08590555555555557),
 ('mahmudullah', 0.07007692307692306),
 ('taskin ahmed', 0.046271428571428574),
 ('rubel hossain', 0.02547333333333333),
 ('najmul hossain shanto', 0.02205),
 ('nasir hossain', -0.016720000000000002),
 ('anamul haque', -0.14991333333333337)]

In [20]:
newdict = dict()
for x,y in sorted_list:
    newdict[y] = x

In [21]:
val_list = [x for y,x in sorted_list ]

## Selected players and their points

In [22]:
for x in val_list[:11]:
    print (newdict[x], x)

mashrafe mortaza 0.22858319999999988
mehidy hasan 0.1963518518518519
mohammad saifuddin 0.15109333333333336
soumya sarkar 0.15051882352941176
imrul kayes 0.13948260869565218
mohammad mithun 0.12819142857142862
tamim iqbal 0.11337163120567378
mustafizur rahman 0.11148928571428572
liton das 0.10149651162790697
mushfiqur rahim 0.09308783783783783
shakib al hasan 0.08590555555555557


In [11]:
base_db = os.path.join(base, 'commentaries')

In [19]:
match_5 = ['1168512','1168513','1168514', '1144487', '1144491']
match_10 = ['1168512','1168513','1168514', '1144487', '1144491', '1168509', '1153844', '1153845', '1153846', '1153316']
match_20 = ['1168512','1168513','1168514', '1144487', '1144491', '1168509', '1153844', '1153845', '1153846', '1153316', '1153315', '1153314', '1153635', '1153636', '1153637', '1153255', '1153254', '1153252', '1153249', '1153248']

In [37]:
match_20_dir = []
for ent in match_20:
    match_20_dir.append(os.path.join(base_db, ent))
   

In [36]:
match_10_dir = []
for ent in match_10:
    match_10_dir.append(os.path.join(base_db, ent))
   

In [35]:
match_5_dir = []
for ent in match_5:
    match_5_dir.append(os.path.join(base_db, ent))
   

In [27]:
os.chdir(base)

In [38]:
match_10_dir


['/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168513',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168514',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1144487',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1144491',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168509',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153844',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153845',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153846',
 '/mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153316']

In [55]:
personal_comm = dict()
for match in match_5_dir:
    os.chdir(match);
    #match_details = get_match_details(match.name)
    print ("moving to "+match);
    innings = [f.path for f in os.scandir(match) if f.is_dir()]
    for single_innings in innings:
        os.chdir(single_innings);
        print ("moving to "+single_innings);
        for jsonfile in os.listdir():
            if 'comm_part' in jsonfile:
                print("opening " + jsonfile)
                with io.open(jsonfile, 'r') as file:
                    data = json.load(file)
                    get_player_score_fromjson(data)
                    
#                     print(metadata)
personal_comm

moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512/innings01
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part13.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part12.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512/innings02
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part12.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links 

{'soumya sarkar': [[0.7089,
   'Bangladesh: 1 Tamim Iqbal, 2 Soumya Sarkar, 3 Shakib Al Hasan, 4 Mushfiqur Rahim (wk), 5 Mohammad Mithun, 6 Mahmudullah, 7 Sabbir Rahman, 8 Mashrafe Mortaza, 9 Mehidy Hasan Miraz, 10 Abu Jayed (ODI debut), 11 Mustafizur Rahman\n\nWest Indies: 1 Shai Hope, 2 Sunil Ambris, 3 Darren Bravo, 4 Roston Chase, 5 Jonathan Carter, 6 Jason Hodler, 7 Fabian Allen, 8 Ashley Nurse, 9 Sheldon Cottrell, 10 Kemar Roach, 11 Raymon Reifer (ODI debut)\n\nMohammad Isam says: "It is quite interesting to see the Bangladesh team management pick Abu Jayed, who was, a couple of days ago, being considered to be replaced by Taskin Ahmed in Bangladesh\'s main World Cup squad.'],
  [-0.6236,
   'Soumya Sarkar to bowl his gentle medium.Seth: "@Hasib Zubayer No offence, but I totally disagree.'],
  [-0.6236,
   'Soumya Sarkar to bowl his gentle medium.Seth: "@Hasib Zubayer No offence, but I totally disagree.'],
  [-0.3182,
   'TQ: "Confused as to why they are making Sarkar bowl here an

In [56]:
result = dict()
# sumofpoints = 0
for key in personal_comm.keys():
    sumofpoints = 0
    for item in personal_comm[key]:
        sumofpoints += item[0]
#     print (key, sumofpoints, len(personal_comm[key]))
    result[key] = sumofpoints/len(personal_comm[key])

In [57]:
import operator
sorted_list = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [58]:
newdict = dict()
for x,y in sorted_list:
    newdict[y] = x

In [59]:
val_list = [x for y,x in sorted_list ]

## Selected players and their points for 5 matches

In [60]:
for x in val_list[:11]:
    print (newdict[x], x)

liton das 0.31024166666666664
mahmudullah 0.2932166666666666
mashrafe mortaza 0.2873956521739131
mehidy hasan 0.18074444444444446
mohammad mithun 0.16011666666666666
mushfiqur rahim 0.14325882352941177
tamim iqbal 0.1412190476190476
najmul hossain shanto 0.128
shakib al hasan 0.12477352941176469
mohammad saifuddin 0.10649999999999998
mustafizur rahman 0.10515


In [61]:
personal_comm = dict()
for match in match_10_dir:
    os.chdir(match);
    #match_details = get_match_details(match.name)
    print ("moving to "+match);
    innings = [f.path for f in os.scandir(match) if f.is_dir()]
    for single_innings in innings:
        os.chdir(single_innings);
        print ("moving to "+single_innings);
        for jsonfile in os.listdir():
            if 'comm_part' in jsonfile:
                print("opening " + jsonfile)
                with io.open(jsonfile, 'r') as file:
                    data = json.load(file)
                    get_player_score_fromjson(data)
                    
#                     print(metadata)
personal_comm

moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512/innings01
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part13.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part12.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512/innings02
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part12.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links 

opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part13.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part12.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153316/innings02
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part9.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part8.json


{'soumya sarkar': [[0.7089,
   'Bangladesh: 1 Tamim Iqbal, 2 Soumya Sarkar, 3 Shakib Al Hasan, 4 Mushfiqur Rahim (wk), 5 Mohammad Mithun, 6 Mahmudullah, 7 Sabbir Rahman, 8 Mashrafe Mortaza, 9 Mehidy Hasan Miraz, 10 Abu Jayed (ODI debut), 11 Mustafizur Rahman\n\nWest Indies: 1 Shai Hope, 2 Sunil Ambris, 3 Darren Bravo, 4 Roston Chase, 5 Jonathan Carter, 6 Jason Hodler, 7 Fabian Allen, 8 Ashley Nurse, 9 Sheldon Cottrell, 10 Kemar Roach, 11 Raymon Reifer (ODI debut)\n\nMohammad Isam says: "It is quite interesting to see the Bangladesh team management pick Abu Jayed, who was, a couple of days ago, being considered to be replaced by Taskin Ahmed in Bangladesh\'s main World Cup squad.'],
  [-0.6236,
   'Soumya Sarkar to bowl his gentle medium.Seth: "@Hasib Zubayer No offence, but I totally disagree.'],
  [-0.6236,
   'Soumya Sarkar to bowl his gentle medium.Seth: "@Hasib Zubayer No offence, but I totally disagree.'],
  [-0.3182,
   'TQ: "Confused as to why they are making Sarkar bowl here an

In [62]:
result = dict()
# sumofpoints = 0
for key in personal_comm.keys():
    sumofpoints = 0
    for item in personal_comm[key]:
        sumofpoints += item[0]
#     print (key, sumofpoints, len(personal_comm[key]))
    result[key] = sumofpoints/len(personal_comm[key])

In [63]:
import operator
sorted_list = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [64]:
newdict = dict()
for x,y in sorted_list:
    newdict[y] = x

In [65]:
val_list = [x for y,x in sorted_list ]

## Selected players and their points for 10 matches

In [66]:
for x in val_list[:11]:
    print (newdict[x], x)

mahmudullah 0.2513285714285714
liton das 0.239255
mohammad saifuddin 0.20964375000000002
mohammad mithun 0.18538947368421055
soumya sarkar 0.16766249999999994
rubel hossain 0.16064736842105265
mashrafe mortaza 0.15791707317073173
mehidy hasan 0.1493909090909091
najmul hossain shanto 0.128
shakib al hasan 0.11238125
tamim iqbal 0.10701749999999999


In [67]:
personal_comm = dict()
for match in match_20_dir:
    os.chdir(match);
    #match_details = get_match_details(match.name)
    print ("moving to "+match);
    innings = [f.path for f in os.scandir(match) if f.is_dir()]
    for single_innings in innings:
        os.chdir(single_innings);
        print ("moving to "+single_innings);
        for jsonfile in os.listdir():
            if 'comm_part' in jsonfile:
                print("opening " + jsonfile)
                with io.open(jsonfile, 'r') as file:
                    data = json.load(file)
                    get_player_score_fromjson(data)
                    
#                     print(metadata)
personal_comm

moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512/innings01
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part13.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part12.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1168512/innings02
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part12.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links 

opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part13.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part12.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153316/innings02
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part9.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153315
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153315/innings01
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening com

opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part10.json
opening comm_part12.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153249/innings02
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part9.json
opening comm_part3.json
opening comm_part2.json
opening comm_part4.json
opening comm_part6.json
opening comm_part8.json
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153248
moving to /mnt/9439c024-e4c1-4178-81f7-44a8e3f2c862/links (git)/commentary-analysis/commentaries/1153248/innings01
opening comm_part1.json
opening comm_part5.json
opening comm_part7.json
opening comm_part11.json
opening comm_part9.json
opening comm_part13.json
opening comm_part3.json
opening comm

{'soumya sarkar': [[0.7089,
   'Bangladesh: 1 Tamim Iqbal, 2 Soumya Sarkar, 3 Shakib Al Hasan, 4 Mushfiqur Rahim (wk), 5 Mohammad Mithun, 6 Mahmudullah, 7 Sabbir Rahman, 8 Mashrafe Mortaza, 9 Mehidy Hasan Miraz, 10 Abu Jayed (ODI debut), 11 Mustafizur Rahman\n\nWest Indies: 1 Shai Hope, 2 Sunil Ambris, 3 Darren Bravo, 4 Roston Chase, 5 Jonathan Carter, 6 Jason Hodler, 7 Fabian Allen, 8 Ashley Nurse, 9 Sheldon Cottrell, 10 Kemar Roach, 11 Raymon Reifer (ODI debut)\n\nMohammad Isam says: "It is quite interesting to see the Bangladesh team management pick Abu Jayed, who was, a couple of days ago, being considered to be replaced by Taskin Ahmed in Bangladesh\'s main World Cup squad.'],
  [-0.6236,
   'Soumya Sarkar to bowl his gentle medium.Seth: "@Hasib Zubayer No offence, but I totally disagree.'],
  [-0.6236,
   'Soumya Sarkar to bowl his gentle medium.Seth: "@Hasib Zubayer No offence, but I totally disagree.'],
  [-0.3182,
   'TQ: "Confused as to why they are making Sarkar bowl here an

In [68]:
result = dict()
# sumofpoints = 0
for key in personal_comm.keys():
    sumofpoints = 0
    for item in personal_comm[key]:
        sumofpoints += item[0]
#     print (key, sumofpoints, len(personal_comm[key]))
    result[key] = sumofpoints/len(personal_comm[key])

In [69]:
import operator
sorted_list = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [70]:
newdict = dict()
for x,y in sorted_list:
    newdict[y] = x

In [71]:
val_list = [x for y,x in sorted_list ]

## Selected players and their points for 20 matches

In [72]:
for x in val_list[:11]:
    print (newdict[x], x)

mohammad saifuddin 0.2078925925925926
mashrafe mortaza 0.17742087912087914
mohammad mithun 0.17130689655172415
soumya sarkar 0.1505188235294117
mehidy hasan 0.1397866666666667
imrul kayes 0.13530298507462693
mahmudullah 0.1307764705882353
liton das 0.11778250000000005
mustafizur rahman 0.0867653846153846
tamim iqbal 0.08457631578947371
rubel hossain 0.07272121212121212
